In [2]:
from pathlib import Path

ROOT = Path.cwd()  # notebook a


In [2]:
# STEP1 – Preprocess
from step1_preprocess.step1_preprocess_pdf import main as preprocess_pdf

preprocess_pdf(
    file_path=ROOT / "step1_preprocess/cv.pdf",
    output_dir=ROOT / "step1_preprocess/step1_output",
    poppler_path=None
)


[STEP 1] Processed 1 pages → /home/taki/Documents/github/ocr_pipeline/step1_preprocess/step1_output


In [3]:
# STEP2 – OCR
from step2_ocr.step2_ocr_paddle import run_ocr

run_ocr(
    input_dir=ROOT / "step1_preprocess/step1_output",
    output_dir=ROOT / "step2_ocr/step2_output"
)


[STEP 2] OCR via Docker (GPU)
  Language: fr
  Input   : /home/taki/Documents/github/ocr_pipeline/step1_preprocess/step1_output
  Output  : /home/taki/Documents/github/ocr_pipeline/step2_ocr/step2_output


W0109 02:03:09.465195     7 init.cc:179] Compiled with WITH_GPU, but no GPU found in runtime.
/usr/local/lib/python3.7/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.7/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/

[STEP 2] OCR completed successfully


In [5]:
# STEP3 – Light Clean OCR JSON
from step3_lightclean.step3_light_clean import clean_ocr_json

clean_ocr_json(
    ROOT / "step2_ocr/step2_output/cv_ocr_raw.json",
    ROOT / "step3_lightclean/step3_output/cv_ocr_cleaned.json"
)


[STEP 3] Cleaned OCR saved → /home/taki/Documents/github/ocr_pipeline/step3_lightclean/step3_output/cv_ocr_cleaned.json


[{'image': 'processed_1.png',
  'lines': [{'text': 'EXPERIENCES/PROJETS',
    'confidence': 0.9644869565963745,
    'bbox': [[1387.0, 134.0],
     [2122.0, 142.0],
     [2121.0, 194.0],
     [1386.0, 186.0]]},
   {'text': 'APPRENTISSAGEENDATASCIENCE-AUTOFORMATION/',
    'confidence': 0.9926514029502869,
    'bbox': [[1034.0, 295.0],
     [2330.0, 295.0],
     [2330.0, 332.0],
     [1034.0, 332.0]]},
   {'text': 'PROJETS PERSONNELS',
    'confidence': 0.9805386662483215,
    'bbox': [[1031.0, 343.0],
     [1550.0, 343.0],
     [1550.0, 380.0],
     [1031.0, 380.0]]},
   {'text': '2o24-Present',
    'confidence': 0.9561295509338379,
    'bbox': [[1023.0, 395.0],
     [1417.0, 391.0],
     [1417.0, 432.0],
     [1023.0, 436.0]]},
   {'text': 'Formation en Data Science et Machine LearningPython,Pandas)',
    'confidence': 0.961251974105835,
    'bbox': [[1046.0, 451.0],
     [2353.0, 451.0],
     [2353.0, 500.0],
     [1046.0, 500.0]]},
   {'text': 'Projets pratiques pour appliquer les com

In [4]:
# STEP4 – RAG Preparation
from step4_rag.step4_rag_prepare import main as prepare_rag_blocks

prepare_rag_blocks(
    ROOT / "step3_lightclean/step3_output/cv_ocr_cleaned.json",
    ROOT / "step4_rag/step4_output/cv_rag_blocks.json",
    doc_id="cv_001"
)


[STEP 4] RAG blocks saved → /home/taki/Documents/github/ocr_pipeline/step4_rag/step4_output/cv_rag_blocks.json
[STEP 4] Total blocks: 2


[{'doc_id': 'cv_001',
  'page': 1,
  'block_id': 0,
  'text': "EXPERIENCES/PROJETS APPRENTISSAGEENDATASCIENCE-AUTOFORMATION/ PROJETS PERSONNELS 2o24-Present Formation en Data Science et Machine LearningPython,Pandas) Projets pratiques pour appliquer les competences FORMATEUR/RESPONSABLEPEDAGOGIQUE-SUP'CENTER, MARRAKECH 2019-2022 Soutien en eco internationale/developpement et informatique de gestion NAJI Enseignement en ligne et presentiel Gestion des reseaux sociaux etcontenus graphiques TAKIEDDINE Developpement des competences pedagogiques et communication PROJETSFREELANCE/PERSONNEL 2014-2022(approx. Personnalisation et publication d'applications mobiles ECONOMISTE/ANALYSTE -Technicien en Maintenance Informatique Conception,assemblage et revente de composants PC Design graphique et projets numeriques personnels (logos,mini-sites web) DEBAT/COMMUNICATION-CLC OPEN DEBATE CENTER APROPOSDEMOI MARRAKECH 2013-2014 Finaliste et meilleur orateur en anglais Economiste de formation.appliquant -

In [2]:
# STEP5 – Embeddings
from step5_embeddings.step5_embeddings import main as build_embeddings

build_embeddings(
    ROOT / "step4_rag/step4_output/cv_rag_blocks.json",
    ROOT / "step5_embeddings/step5_output"
)


/home/taki/Documents/github/ocr_pipeline/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[STEP 5] Stored 2 embeddings
[STEP 5] Chroma directory: /home/taki/Documents/github/ocr_pipeline/step5_embeddings/step5_output


In [3]:
# STEP6 – Retrieval (DEBUG ONLY)
from step6_retrieval.step6_retrieval import main as retrieve_blocks

results = retrieve_blocks(
    chroma_dir=ROOT / "step5_embeddings/step5_output",
    query="Quelle est l'expérience du candidat en data science ?",
    top_k=3
)

results



[STEP 6] Retrieved blocks:

Doc: cv_001 | Page: 1
EXPERIENCES/PROJETS APPRENTISSAGEENDATASCIENCE-AUTOFORMATION/ PROJETS PERSONNELS 2o24-Present Formation en Data Science et Machine LearningPython,Pandas) Projets pratiques pour appliquer les competences FORMATEUR/RESPONSABLEPEDAGOGIQUE-SUP'CENTER, MARRAKECH 2019-2022 Soutien en eco internationale/de 

Doc: cv_001 | Page: 1
2016-Juillet2018 LICENCE FONDAMENTALEENECONOMIE ET GESTION-FSJES Arabe:Langue maternelle MARRAKECH 2013-Juillet2015 FrancaisCourant DEUGENECONOMIE ETGESTION-FSJES MARRAKECH AnglaisAvance 2011-Juillet2013 BACCALAUREAT TGC(TECHNIQUE DE GESTION COMPTABLE) LYCEE CHARIF AL EDRISSI,SAFI CONTACT 2009-Juill 



{'ids': [['block_0', 'block_1']],
 'embeddings': None,
 'documents': [["EXPERIENCES/PROJETS APPRENTISSAGEENDATASCIENCE-AUTOFORMATION/ PROJETS PERSONNELS 2o24-Present Formation en Data Science et Machine LearningPython,Pandas) Projets pratiques pour appliquer les competences FORMATEUR/RESPONSABLEPEDAGOGIQUE-SUP'CENTER, MARRAKECH 2019-2022 Soutien en eco internationale/developpement et informatique de gestion NAJI Enseignement en ligne et presentiel Gestion des reseaux sociaux etcontenus graphiques TAKIEDDINE Developpement des competences pedagogiques et communication PROJETSFREELANCE/PERSONNEL 2014-2022(approx. Personnalisation et publication d'applications mobiles ECONOMISTE/ANALYSTE -Technicien en Maintenance Informatique Conception,assemblage et revente de composants PC Design graphique et projets numeriques personnels (logos,mini-sites web) DEBAT/COMMUNICATION-CLC OPEN DEBATE CENTER APROPOSDEMOI MARRAKECH 2013-2014 Finaliste et meilleur orateur en anglais Economiste de formation.app

In [2]:

# STEP7 – LLM Answering
from step7_llm.step7_llm_answering import answer_question

answer = answer_question(
    chroma_dir=ROOT / "step5_embeddings/step5_output",
    question="Quelle est l'expérience du candidat en data science ?",
    top_k=3
)

print(answer)


/home/taki/Documents/github/ocr_pipeline/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



[STEP 6] Retrieved blocks:

Doc: cv_001 | Page: 1
EXPERIENCES/PROJETS APPRENTISSAGEENDATASCIENCE-AUTOFORMATION/ PROJETS PERSONNELS 2o24-Present Formation en Data Science et Machine LearningPython,Pandas) Projets pratiques pour appliquer les competences FORMATEUR/RESPONSABLEPEDAGOGIQUE-SUP'CENTER, MARRAKECH 2019-2022 Soutien en eco internationale/de 

Doc: cv_001 | Page: 1
2016-Juillet2018 LICENCE FONDAMENTALEENECONOMIE ET GESTION-FSJES Arabe:Langue maternelle MARRAKECH 2013-Juillet2015 FrancaisCourant DEUGENECONOMIE ETGESTION-FSJES MARRAKECH AnglaisAvance 2011-Juillet2013 BACCALAUREAT TGC(TECHNIQUE DE GESTION COMPTABLE) LYCEE CHARIF AL EDRISSI,SAFI CONTACT 2009-Juill 

Selon le contexte fourni, l'expérience du candidat en data science inclut:

- Projets pratiques pour appliquer les compétences acquises lors de la formation en Data Science et Machine Learning avec Python et Pandas (2024 - Present).

- Certificats obtenus : IBM Data Science Professional Certificate (validité jusqu'en 20

In [1]:
from pathlib import Path

ROOT = Path.cwd()  # notebook a


# batch file to run the entire pipeline at once (1 - 5)

!python /home/taki/Documents/github/ocr_pipeline/caller_batch/caller_batch.py



=== Processing CV: cv3 ===
[STEP 1] Processed 1 pages → /home/taki/Documents/github/ocr_pipeline/caller_batch/step1_batch_output/cv3
[STEP 2] OCR via Docker
  Language: fr
  Input   : /home/taki/Documents/github/ocr_pipeline/caller_batch/step1_batch_output/cv3
  Output  : /home/taki/Documents/github/ocr_pipeline/caller_batch/step2_batch_output/cv3
W0109 02:17:12.829255     7 init.cc:179] Compiled with WITH_GPU, but no GPU found in runtime.
/usr/local/lib/python3.7/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keywo

In [2]:
from pathlib import Path
import sys

# =====================================================
# PROJECT ROOT
# =====================================================

ROOT = Path(__file__).resolve().parent.parent
sys.path.append(str(ROOT))

# =====================================================
# CONFIG
# =====================================================
CV_INPUT_DIR = ROOT / "caller_batch" / "cvs_input"

BASE_STEP1_OUT = ROOT / "caller_batch" / "step1_batch_output"
BASE_STEP2_OUT = ROOT / "caller_batch" / "step2_batch_output"
BASE_STEP3_OUT = ROOT / "caller_batch" / "step3_batch_output"
BASE_STEP4_OUT = ROOT / "caller_batch" / "step4_batch_output"
BASE_STEP5_OUT = ROOT / "caller_batch" / "step5_batch_output"


# =====================================================
# IMPORTS (package-based)
# =====================================================
from step1_preprocess.step1_preprocess_pdf import main as preprocess_file
from step2_ocr.step2_ocr_paddle import run_ocr
from step3_lightclean.step3_light_clean import clean_ocr_json
from step4_rag.step4_rag_prepare import main as prepare_rag_blocks
from step5_embeddings.step5_embeddings import main as build_embeddings

# =====================================================
# BATCH PIPELINE
# =====================================================
VALID_EXTENSIONS = {".pdf", ".png", ".jpg", ".jpeg"}

for file_path in CV_INPUT_DIR.iterdir():
    if file_path.suffix.lower() not in VALID_EXTENSIONS:
        continue

    cv_name = file_path.stem
    print(f"\n=== Processing CV: {cv_name} ===")

    # Per-CV folders
    step1_out = BASE_STEP1_OUT / cv_name
    step2_out = BASE_STEP2_OUT / cv_name
    step3_out = BASE_STEP3_OUT / cv_name
    step4_out = BASE_STEP4_OUT / cv_name
    step5_out = BASE_STEP5_OUT / cv_name

    step5_out.mkdir(parents=True, exist_ok=True)

    # -------------------------------
    # Step 1 — Preprocess
    # -------------------------------
    preprocess_file(
        file_path=file_path,
        output_dir=step1_out,
        poppler_path=None  # Ubuntu
    )

    # -------------------------------
    # Step 2 — OCR (GPU)
    # -------------------------------
    try:
        run_ocr(step1_out, step2_out)
    except Exception as e:
        print(f"[ERROR] Step 2 failed: {e}")
        continue

    raw_json = step2_out / "cv_ocr_raw.json"
    if not raw_json.exists():
        print("[SKIP] No OCR output")
        continue

    # -------------------------------
    # Step 3 — Light clean
    # -------------------------------
    clean_json = step3_out / "cv_ocr_cleaned.json"
    step3_out.mkdir(parents=True, exist_ok=True)

    try:
        clean_ocr_json(raw_json, clean_json)
    except Exception as e:
        print(f"[ERROR] Step 3 failed: {e}")
        continue

    # -------------------------------
    # Step 4 — RAG blocks
    # -------------------------------
    rag_blocks = step4_out / "cv_rag_blocks.json"
    step4_out.mkdir(parents=True, exist_ok=True)

    prepare_rag_blocks(
        input_path=clean_json,
        output_path=rag_blocks,
        doc_id=cv_name
    )

    # -------------------------------
    # Step 5 — Embeddings (GPU)
    # -------------------------------
    build_embeddings(
        blocks_path=rag_blocks,
        chroma_dir=step5_out
    )


NameError: name '__file__' is not defined